# 数据预处理

In [ ]:
import numpy as np
import torch

time_step = 10
max_time = 5400

content = np.loadtxt("Data\students003.txt")
max_id = int(np.max(content[:, 1]))
max_time = int(np.max(content[:, 0]) / time_step) + 1

t_id_fxy = np.zeros((max_time, max_id, 2))

for l in content:
    t_id_fxy[int(l[0] / time_step), int(l[1] - 1), :] = l[-2:]

t_id_fxy = torch.from_numpy(t_id_fxy).float()
delta_xy = t_id_fxy[1:] - t_id_fxy[:-1]
delta_xy = torch.cat([torch.zeros(1, max_id, 2), delta_xy])
zero_idx = torch.argwhere(t_id_fxy[:, :, 0] == 0)
for z_idx in zero_idx:
    t, i = z_idx
    delta_xy[t, i, :] = 0
    if t < int(max_time / time_step):
        delta_xy[t + 1, i, :] = 0

t_id_fxy_vxvy = torch.cat([t_id_fxy, delta_xy], dim=-1)
torch.save(t_id_fxy_vxvy, "Data\\track.pt")

# 聚类

## k_means 聚类

In [1]:
import torch
import cv2
from src.clustering import k_means
from src.draw import Drawer

track = torch.load("Data\\track.pt")
max_x = torch.max(track[:, :, 0])
min_x = torch.min(track[:, :, 0])
max_y = torch.max(track[:, :, 1])
min_y = torch.min(track[:, :, 1])

k = 5
d = Drawer(min_x, max_x, min_y, max_y)

video_output = 'Videos\\kmeans_1.mp4'
fps=30
video = cv2.VideoWriter(video_output, 0, fps, (420, 380))

for i in range(track.shape[0]):
    t = track[i]
    idx = torch.argwhere(t[:, 0] != 0)
    idx = idx[:, 0]
    t = t[idx]
    t = t[:, :2]
    clu = k_means(t, k)

    canvas = d.draw(clu)
    video.write(canvas)

video.release()

In [3]:
import torch
import cv2
from sklearn.cluster import KMeans
from src.draw import Drawer, org_data
import warnings

warnings.filterwarnings("ignore")

track = torch.load("Data\\track.pt")
max_x = torch.max(track[:, :, 0])
min_x = torch.min(track[:, :, 0])
max_y = torch.max(track[:, :, 1])
min_y = torch.min(track[:, :, 1])

k = 5
d = Drawer(min_x, max_x, min_y, max_y)
kmeans = KMeans(n_clusters=k, n_init=5)

video_output = "Videos\\kmeans_2.mp4"
fps = 30
video = cv2.VideoWriter(video_output, 0, fps, (420, 380))

for i in range(track.shape[0]):
    t = track[i]
    idx = torch.argwhere(t[:, 0] != 0)
    idx = idx[:, 0]
    t = t[idx]
    t = t[:, :2]
    kmeans.fit(t)
    clu = org_data(t, kmeans.labels_)
    canvas = d.draw(clu)

    video.write(canvas)
video.release()

## DBSCAN 聚类

In [4]:
import torch
import cv2
from sklearn.cluster import DBSCAN
from src.draw import Drawer, org_data

track = torch.load("Data\\track.pt")
max_x = torch.max(track[:, :, 0])
min_x = torch.min(track[:, :, 0])
max_y = torch.max(track[:, :, 1])
min_y = torch.min(track[:, :, 1])

db = DBSCAN(eps=2, min_samples=1)
d = Drawer(min_x, max_x, min_y, max_y)

video_output = "Videos\\DBSAN.mp4"
fps = 30
video = cv2.VideoWriter(video_output, 0, fps, (420, 380))

for i in range(track.shape[0]):
    t = track[i]
    idx = torch.argwhere(t[:, 0] != 0)
    idx = idx[:, 0]
    t = t[idx]
    t = t[:, :2]
    db.fit(t)
    clu = org_data(t, db.labels_)
    canvas = d.draw(clu)
    video.write(canvas)
video.release()

## 层次聚类

In [5]:
import torch
import cv2
from sklearn.cluster import AgglomerativeClustering
from src.draw import Drawer, org_data

track = torch.load("Data\\track.pt")
max_x = torch.max(track[:, :, 0])
min_x = torch.min(track[:, :, 0])
max_y = torch.max(track[:, :, 1])
min_y = torch.min(track[:, :, 1])

k = 5
clustering = AgglomerativeClustering(n_clusters=5, linkage="ward")
d = Drawer(min_x, max_x, min_y, max_y)


video_output = "Videos\\AGNES.mp4"
fps = 30
video = cv2.VideoWriter(video_output, 0, fps, (420, 380))

for i in range(track.shape[0]):
    t = track[i]
    idx = torch.argwhere(t[:, 0] != 0)
    idx = idx[:, 0]
    t = t[idx]
    t = t[:, :2]
    clustering.fit(t)
    clu = org_data(t, clustering.labels_)
    canvas = d.draw(clu)
    video.write(canvas)
video.release()

: 